In [2]:
import pandas as pd
import numpy as np

import requests

In [3]:
frost_id = '23b60ca7-860f-4c84-bb2a-431206a0681d'
frost_secert = '4fbc591a-214d-4066-a7a2-d268cf98f3eb'


In [4]:
def get_json(endpoint,params = {},clint_id = '23b60ca7-860f-4c84-bb2a-431206a0681d'):
    r = requests.get(endpoint, params, auth=(clint_id,''))
    json = r.json()
    if r.status_code == 200:
        data = json['data']
        print('Data retrieved from frost.met.no')
    else:
        print('Error! Returned status code %s' % r.status_code)
        print('Message: %s' % json['error']['message'])
        print('Reason: %s' % json['error']['reason'])
    return(data)


def get_df(endpoint,params = {},client_id = '23b60ca7-860f-4c84-bb2a-431206a0681d'):
    data = get_json(endpoint,params,client_id)
    df = pd.DataFrame()
    for i in range(len(data)):
        row = pd.DataFrame(data[i]['observations'])
        row['referenceTime'] = data[i]['referenceTime']
        row['sourceId'] = data[i]['sourceId']
        df = df.append(row)

    return(df.reset_index())
    

def get_df_row(json):
    
    df_row = pd.io.json.json_normalize(json)
    
    return(df_row)
    
    

In [5]:
### EXAMPLE
# Define endpoint and parameters
url2 = 'https://frost.met.no/observations/v0.jsonld'
parameters = {
    'sources': 'SN18700,SN90450',
    'elements': 'mean(air_temperature P1D),sum(precipitation_amount P1D),mean(wind_speed P1D)',
    'referencetime': '2010-04-01/2010-04-03',
}
# Issue an HTTP GET request
r = requests.get(endpoint, parameters, auth=(frost_id,''))
# Extract JSON data
json = r.json()

NameError: name 'endpoint' is not defined

In [6]:
cities = ['oslo','bergen','trondheim','moi','oppsal']

json_list = []
df_list = []

for city in cities:
    city_json = get_json('https://frost.met.no/locations/v0.jsonld?names={}'.format(city))
    json_list.append(city_json)
    df_list.append(get_df_row(city_json))
    
df = pd.concat(df_list)
df.reset_index(inplace=True,drop=True)
    

Data retrieved from frost.met.no
Data retrieved from frost.met.no
Data retrieved from frost.met.no
Data retrieved from frost.met.no
Data retrieved from frost.met.no


In [99]:
df_list = []
elements = {}

for coord in df['geometry.coordinates']:
    coord_str = str(coord[0]) + ', ' + str(coord[1])
    print(coord_str)
    params = {
        'elements': elements
    }
    coord_json = get_json('https://frost.met.no/sources/v0.jsonld',params)



10.746092, 59.912728
Data retrieved from frost.met.no
5.329072, 60.398258
Data retrieved from frost.met.no
10.395058, 63.430483


KeyboardInterrupt: 

In [7]:
station_json = get_json('https://frost.met.no/sources/v0.jsonld?country=Norge')
station_df = get_df_row(station_json)
station_df = station_df[~station_df['geometry.coordinates'].isnull()]

Data retrieved from frost.met.no


In [20]:
station_df.to_csv('../csv/stations.csv')